# Assistant API 테스트 노트북

이 노트북은 OpenAI Assistant API와 Chat Completion API를 사용한 스토리 생성 및 검색 기능을 테스트합니다.

In [1]:
# 필요한 라이브러리 임포트
import sys
import os
import json
import time
from dotenv import load_dotenv
from openai import OpenAI
from app.config import Config
# 현재 디렉토리를 시스템 경로에 추가
sys.path.append(os.path.abspath('..'))

# 환경 변수 로드
load_dotenv()

# OpenAI 클라이언트 초기화
client = OpenAI()

GPT-4o Assistant retrieved: asst_Mp2a52hDvIkr0oEk7xdqiKnA
Assistants initialized successfully


## 1. Assistant API 초기화 및 확인

In [2]:
# 환경 변수에서 Assistant ID 가져오기
gpt4o_assistant_id = os.getenv('GPT4O_ASSISTANT_ID', '')

# Assistant ID가 있으면 조회, 없으면 새로 생성
if gpt4o_assistant_id:
    try:
        gpt4o_assistant = client.beta.assistants.retrieve(gpt4o_assistant_id)
        print(f"GPT-4o Assistant retrieved: {gpt4o_assistant.id}")
    except Exception as e:
        print(f"Error retrieving GPT-4o Assistant: {str(e)}")
        gpt4o_assistant = None
else:
    gpt4o_assistant = None

# Assistant가 없으면 새로 생성
if gpt4o_assistant is None:
    # 시스템 프롬프트 정의
    HYBRID_SYSTEM_PROMPT = """당신은 고전 문학 전문 작가입니다. 주어진 내용을 기반으로 더 풍부하고 자세한 내용의 이야기를 생성해주세요.

다음 지침을 반드시 따라주세요:
1. 고전 문학의 특징적인 표현과 문체를 사용해주세요.
2. 모든 외래어와 현대적 표현은 주어진 내용이더라도 예외 없이 한자어나 순우리말로 바꾸어야 합니다.
   변환 원칙:
   - 현대 과학용어는 천지조화/자연의 이치 관련 한자어로 변환
   - 괴물/요정 등은 귀신/요괴/괴이한 것 등 전통적 초자연 존재로 변환
   - 현대 직업/역할은 그에 준하는 전통 직업/역할로 변환
   - 현대 사회 용어는 그 본질적 의미를 담은 한자어로 변환
   
   예시:
   - 하이틴 로맨스 → 꽃다운 나이의 사랑 이야기
   - 빌런 → 악인
3. 판타지적이거나 비현실적인 요소는 처음 등장할 때 그 특성을 상세히 설명한 후, 이후에는 정해진 한자어나 순우리말로 표현하세요.
   설명 원칙:
   - 그 존재/현상의 본질적 특성을 먼저 서술
   - 전통적인 귀신/요괴 명명법을 따라 적절한 한자어 작명
   - 이후 그 한자어 명칭을 일관되게 사용
   
   예시:
   - 좀비 → "기괴한 몰골에 악취를 풍기고 짐승 소리를 내며 인육을 탐하는 자들이 나타났다. 특히 사람의 피에 민감히 반응하니, 이는 곧 생사역(生死疫)이라 불리는 괴질이라." 설명 후 '생사역 걸린 자'로 지속 사용
4. 시대적 배경과 분위기를 자연스럽게 반영해주세요.
5. 새로운 사건이나 인물을 추가하지 말고, 기존 내용을 더 풍부하게 표현해주세요.
6. 인물의 심리 묘사, 배경 설명, 상황 묘사를 더 상세하게 추가해주세요.
7. 문장 간의 자연스러운 연결과 흐름을 유지해주세요.
8. 원본 내용의 핵심 내용, 인물, 사건, 감정을 유지하면서 확장해주세요.

결과물은 하나의 이야기로 작성해주세요.
"""
    
    # Assistant 생성
    gpt4o_assistant = client.beta.assistants.create(
        name="Story Expander",
        instructions=HYBRID_SYSTEM_PROMPT,
        model=Config.GPT_4O_MODEL
    )
    print(f"New GPT-4o Assistant created: {gpt4o_assistant.id}")
    print(f"GPT4O_ASSISTANT_ID={gpt4o_assistant.id}")

# Assistant 정보 출력
print(f"\nAssistant Name: {gpt4o_assistant.name}")
print(f"Assistant Model: {gpt4o_assistant.model}")

GPT-4o Assistant retrieved: asst_Mp2a52hDvIkr0oEk7xdqiKnA

Assistant Name: Story Expander
Assistant Model: gpt-4o-2024-11-20


## 2. 기본 스토리 생성 테스트 (Chat Completion API)

In [3]:
def generate_base_story(theme, tags={}):
    """파인튜닝된 모델로 기본 스토리 생성"""
    # 키워드 추출
    keywords = extract_keywords(theme)
    
    # 프롬프트 포맷팅
    STORY_GENERATION_PROMPT = """당신은 다양한 장르의 전문 작가입니다. 제시된 라벨링 기준을 참고하여 이야기를 생성해 주세요.

- 주어진 내용분류, 주제어, 주제문은 생성될 단락의 라벨링입니다.
- 장르와 배경에 맞는 적절한 문체와 표현을 사용해주세요.
- 인물의 행동, 대화, 감정을 자연스럽게 표현해주세요.
- 상황과 인물 관계를 효과적으로 담아주세요."""
    
    # 태그 포맷팅
    tags_str = json.dumps(tags, ensure_ascii=False) if tags else ""
    
    # 파인튜닝된 모델로 스토리 생성
    response = client.chat.completions.create(
        model=Config.FINE_TUNED_MODEL,  # 파인튜닝된 모델 ID
        messages=[
            {"role": "system", "content": STORY_GENERATION_PROMPT},
            {"role": "user", "content": f"[내용 분류]\n{tags_str}\n\n[주제어]\n{keywords}\n\n[주제문]\n{theme}"}
        ],
        temperature=0.7
    )
    
    return response.choices[0].message.content

def extract_keywords(theme):
    """주제문에서 키워드 추출"""
    response = client.chat.completions.create(
        model=Config.GPT_MINI_MODEL,
        messages=[
            {
                "role": "system",
                "content": "당신은 주제문에서 핵심 주제어를 추출하는 전문가입니다. "
                          "주제문을 분석하여 3-10개의 핵심 주제어를 추출해주세요. "
                          "주제어는 쉼표로 구분된 단어나 구문으로 반환해주세요. "
                          "다른 설명이나 부가적인 내용 없이 주제어만 반환해주세요."
            },
            {
                "role": "user",
                "content": theme
            }
        ],
        temperature=0.3
    )
    return response.choices[0].message.content.strip()

# 테스트 실행
test_theme = "젊은 선비가 과거시험을 보러 가는 길에 이상한 여인을 만나 사랑에 빠지는 이야기"
test_tags = {
    "genre": ["로맨스", "판타지"],
    "characters": ["선비", "여인"],
    "emotion": ["사랑", "설렘"]
}

base_story = generate_base_story(test_theme, test_tags)
print("\n=== 기본 스토리 ===\n")
print(base_story)


=== 기본 스토리 ===

대나무로 된 젊은 선비는 운남으로 과거시험을 보러 가는 길에, 한 여자가 갑자기 그 앞에 나타나 마치 물 위를 걷는 듯했다. 선비는 그녀에게 물었고, 여인은 웃으며 "어디로 가는 길입니까?"라고 물었다. 선비는 길에 대해 이야기했고, 여인은 돌아가려 했다. 선비는 그녀가 매우 아름답고, 그녀의 말이 매우 단정하다고 느꼈다. 그래서 그녀를 붙잡아야겠다고 생각했고, 결국 그녀의 아름다운 몸을 사랑하게 되었다.


## 3. Assistant API로 스토리 확장 테스트

In [4]:
# 쓰레드 아이디 출력
response = client.beta.threads.create()
print(f"response: {response}")
print(f"thread id: {response.id}")

response: Thread(id='thread_ogoQrqo2yk3XkDYELMDn1IMc', created_at=1740637466, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))
thread id: thread_ogoQrqo2yk3XkDYELMDn1IMc


In [7]:
THREAD_ID = "thread_ogoQrqo2yk3XkDYELMDn1IMc" # 쓰레드 아이디 고정

In [8]:
def expand_story_with_assistant(base_story):
    """Assistant API를 사용하여 스토리 확장"""
    # Thread 생성
    thread = client.beta.threads.create()
    
    # 메시지 추가
    client.beta.threads.messages.create(
        thread_id=THREAD_ID,
        role="user",
        content=f"다음 이야기를 더 풍부하게 확장해주세요:\n\n{base_story}"
    )
    
    # Run 생성 및 완료 대기
    run = client.beta.threads.runs.create(
        thread_id=THREAD_ID,
        assistant_id=gpt4o_assistant.id
    )
    
    print(f"[DEBUG]: run response: {run}")
    
    # Run 완료 대기
    print("Assistant API 처리 중...", end="")
    while True:
        run_status = client.beta.threads.runs.retrieve(
            thread_id=THREAD_ID,
            run_id=run.id
        )
        if run_status.status == 'completed':
            print(" 완료!")
            break
        elif run_status.status in ['failed', 'cancelled', 'expired']:
            print(f"\nError: {run_status.status}")
            return None
        print(".", end="", flush=True)
        time.sleep(1)
    
    # 결과 메시지 가져오기
    messages = client.beta.threads.messages.list(
        thread_id=THREAD_ID
    )
    
    # 마지막 메시지 (Assistant의 응답) 가져오기
    for message in messages.data:
        if message.role == "assistant":
            return message.content[0].text.value
    
    return None

# 테스트 실행
expanded_story = expand_story_with_assistant(base_story)
print("\n=== 확장된 스토리 ===\n")
print(expanded_story)

[DEBUG]: run response: Run(id='run_bD9Y0ugiTzekl19atRKYXHH3', assistant_id='asst_Mp2a52hDvIkr0oEk7xdqiKnA', cancelled_at=None, completed_at=None, created_at=1740637638, expires_at=1740638238, failed_at=None, incomplete_details=None, instructions='당신은 고전 문학 전문 작가입니다. 주어진 내용을 기반으로 더 풍부하고 자세한 내용의 이야기를 생성해주세요.\n\n다음 지침을 반드시 따라주세요:\n1. 고전 문학의 특징적인 표현과 문체를 사용해주세요.\n2. 모든 외래어와 현대적 표현은 주어진 내용이더라도 예외 없이 한자어나 순우리말로 바꾸어야 합니다.\n   변환 원칙:\n   - 현대 과학용어는 천지조화/자연의 이치 관련 한자어로 변환\n   - 괴물/요정 등은 귀신/요괴/괴이한 것 등 전통적 초자연 존재로 변환\n   - 현대 직업/역할은 그에 준하는 전통 직업/역할로 변환\n   - 현대 사회 용어는 그 본질적 의미를 담은 한자어로 변환\n   \n   예시:\n   - 하이틴 로맨스 → 꽃다운 나이의 사랑 이야기\n   - 빌런 → 악인\n3. 판타지적이거나 비현실적인 요소는 처음 등장할 때 그 특성을 상세히 설명한 후, 이후에는 정해진 한자어나 순우리말로 표현하세요.\n   설명 원칙:\n   - 그 존재/현상의 본질적 특성을 먼저 서술\n   - 전통적인 귀신/요괴 명명법을 따라 적절한 한자어 작명\n   - 이후 그 한자어 명칭을 일관되게 사용\n   \n   예시:\n   - 좀비 → "기괴한 몰골에 악취를 풍기고 짐승 소리를 내며 인육을 탐하는 자들이 나타났다. 특히 사람의 피에 민감히 반응하니, 이는 곧 생사역(生死疫)이라 불리는 괴질이라." 설명 후 \'생사역 걸린 자\'로 지속 사용\n4. 시대적 배경과 분위기를 자연스럽게 반영해주세요.\n5. 새로운

In [9]:
# 확장된 스토리 다시 조회
thread_messages = client.beta.threads.messages.list(THREAD_ID)
print(thread_messages.data)

[Message(id='msg_5VhX5yBw8G8dDAiKwdfM352X', assistant_id='asst_Mp2a52hDvIkr0oEk7xdqiKnA', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='대나무로 된 젊은 선비는 늠름하고도 인자한 기품을 가진 사내로, 그 이름은 이화(李華)라 하였다. 운남으로 과거를 보러 떠나는 길에, 그는 깊은 산과 울창한 숲길을 지나고 있었다. 바람이 나뭇잎을 스치며 은은히 바람소리를 전하고, 그의 걸음은 무겁지도 가볍지도 않게 절제되어 있었다. 이화는 학문에 진실한 사람이었으나, 아직 젊은 혈기와 낭만적인 마음도 지녔으니, 세상을 헤쳐나가는 열정과 호기심이 그 눈빛 속에는 가득하였다.\n\n그날은 하늘의 기운이 묘하게도 흔들려 구름이 변덕스럽게 움직이더니, 갑작스레 눈앞에 한 여인이 나타났다. 그녀는 마치 허공 속에서 나타난 듯, 물결을 거스르듯이 그 발자국은 흔적이 없었다. 이화는 멈춰서서 그녀를 그윽한 시선으로 주목하였다. 여인의 옷자락은 연못 위의 구름처럼 가볍고 부드러웠으며 얼굴의 고운 빛은 은은한 달빛을 닮았다. 그녀는 선명하게 빛나는 눈동자로 이화를 마주하다가, 입가에 잔잔한 미소를 머금고 입을 열었다.\n\n“대감댁 공자님, 어디로 가시는 길입니까?” 그녀의 목소리는 맑은 계곡물처럼 부드럽고 명확하였다. 이화는 약간의 놀라움과 함께 그녀를 찬찬히 살피더니 대답하였다.\n\n“소승은 운남으로 과거를 치르러 떠나는 정처 없는 나그넵니다. 그런데 여인이 홀로 이 깊은 산 속에 나타나다니, 실로 괴이하게 여겨지는군요. 무슨 연유로 여기 계시는지 묻고 싶습니다.”\n\n여인은 미소를 띠며 그의 물음에 대답하지 않았다. 대신 물끄러미 그의 얼굴을 바라보다가 희미하게 고개를 끄덕이고는 사뿐히 물러섰다. 그녀의 걸음걸이는 마치 물 위를 걷듯이 허공에 묘하게 흩어지며 사라지려 하고 있었다. 그때 이화는

## 4. 제목 및 추천 생성 테스트 (Chat Completion API)

In [10]:
def generate_title_and_recommendations(content, theme):
    """제목과 추천 이야기를 한 번의 API 호출로 생성"""
    completion = client.chat.completions.create(
        model=Config.GPT_MINI_MODEL,
        messages=[
            {"role": "system", "content": """두 가지 작업을 수행해주세요:
            1. 제목 생성: 주어진 고전소설 단락에 어울리는 간략한 제목을 생성하세요. 제목은 13글자를 넘기지 마세요.
            2. 추천 생성: 주어진 주제와 비슷한 새로운 고전소설 줄거리 생성을 위한 이야기 소스를 3개 생성하세요.
               각 추천은 한 문장으로, "~~이야기"로 끝나야 합니다. (예: "모험이 시작되는 이야기")
               영어를 사용하지 마세요.
            
            JSON 형식으로 응답해주세요:
            {
                "title": "생성된 제목",
                "recommendations": ["추천1", "추천2", "추천3"]
            }"""},
            {"role": "user", "content": f"단락: {content}\n\n주제: {theme}"}
        ],
        response_format={"type": "json_object"}
    )
    
    response_text = completion.choices[0].message.content
    response_data = json.loads(response_text)
    
    return response_data

# 테스트 실행
title_and_recommendations = generate_title_and_recommendations(expanded_story, test_theme)
print("\n=== 제목 및 추천 ===\n")
print(f"제목: {title_and_recommendations['title']}")
print("\n추천 이야기:")
for i, rec in enumerate(title_and_recommendations['recommendations'], 1):
    print(f"{i}. {rec}")


=== 제목 및 추천 ===

제목: 운명의 만남

추천 이야기:
1. 선비와 구름의 이야기
2. 젊은 선비의 애정 이야기
3. 운명적 사랑의 여정 이야기


## 5. 전체 프로세스 통합 테스트

In [11]:
def generate_complete_story(theme, tags={}):
    """전체 스토리 생성 프로세스 통합 테스트"""
    print("1. 기본 스토리 생성 중...")
    base_story = generate_base_story(theme, tags)
    print(f"   완료! ({len(base_story)} 글자)\n")
    
    print("2. 스토리 확장 중...")
    expanded_story = expand_story_with_assistant(base_story)
    print(f"   완료! ({len(expanded_story)} 글자)\n")
    
    print("3. 제목 및 추천 생성 중...")
    result = generate_title_and_recommendations(expanded_story, theme)
    print("   완료!\n")
    
    return {
        "base_story": base_story,
        "expanded_story": expanded_story,
        "title": result["title"],
        "recommendations": result["recommendations"]
    }

# 새로운 테마로 테스트
new_theme = "산속 암자에서 수행하던 스님이 오래된 비밀을 간직한 여인을 만나 갈등하는 이야기"
new_tags = {
    "genre": ["역사", "드라마"],
    "characters": ["스님", "여인"],
    "emotion": ["갈등", "고뇌"],
    "location": ["산속", "암자"]
}

result = generate_complete_story(new_theme, new_tags)

# 결과 출력
print("\n=== 최종 결과 ===\n")
print(f"제목: {result['title']}\n")
print("확장된 스토리:")
print(result['expanded_story'][:500] + "...\n")
print("추천 이야기:")
for i, rec in enumerate(result['recommendations'], 1):
    print(f"{i}. {rec}")

1. 기본 스토리 생성 중...
   완료! (466 글자)

2. 스토리 확장 중...
[DEBUG]: run response: Run(id='run_3k1InKVFgnZRSxv0IVPxKdB2', assistant_id='asst_Mp2a52hDvIkr0oEk7xdqiKnA', cancelled_at=None, completed_at=None, created_at=1740638565, expires_at=1740639165, failed_at=None, incomplete_details=None, instructions='당신은 고전 문학 전문 작가입니다. 주어진 내용을 기반으로 더 풍부하고 자세한 내용의 이야기를 생성해주세요.\n\n다음 지침을 반드시 따라주세요:\n1. 고전 문학의 특징적인 표현과 문체를 사용해주세요.\n2. 모든 외래어와 현대적 표현은 주어진 내용이더라도 예외 없이 한자어나 순우리말로 바꾸어야 합니다.\n   변환 원칙:\n   - 현대 과학용어는 천지조화/자연의 이치 관련 한자어로 변환\n   - 괴물/요정 등은 귀신/요괴/괴이한 것 등 전통적 초자연 존재로 변환\n   - 현대 직업/역할은 그에 준하는 전통 직업/역할로 변환\n   - 현대 사회 용어는 그 본질적 의미를 담은 한자어로 변환\n   \n   예시:\n   - 하이틴 로맨스 → 꽃다운 나이의 사랑 이야기\n   - 빌런 → 악인\n3. 판타지적이거나 비현실적인 요소는 처음 등장할 때 그 특성을 상세히 설명한 후, 이후에는 정해진 한자어나 순우리말로 표현하세요.\n   설명 원칙:\n   - 그 존재/현상의 본질적 특성을 먼저 서술\n   - 전통적인 귀신/요괴 명명법을 따라 적절한 한자어 작명\n   - 이후 그 한자어 명칭을 일관되게 사용\n   \n   예시:\n   - 좀비 → "기괴한 몰골에 악취를 풍기고 짐승 소리를 내며 인육을 탐하는 자들이 나타났다. 특히 사람의 피에 민감히 반응하니, 이는 곧 생사역(生死疫)이라 불리는 괴질이라." 설명 후 \'생사역 걸린

## 6. 결과 저장

In [13]:
# 결과를 JSON 파일로 저장
output_file = "test_results.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

print(f"\n결과가 {output_file}에 저장되었습니다.")


결과가 test_results.json에 저장되었습니다.
